List Kriteria<br>
C1: Cara bicara,<br>
C2: Etika,<br>
C3: Semangat,<br>
C4: Permintaan gaji,<br>
C5: Wibawa,<br>
C6: Pengalaman kerja,<br>
C7: Cara berpenampilan,<br>
C8: Besar motivasi untuk diterima,<br>
C9: Perencanaan kehidupan,<br>
C10: Pengetahuan tentang perusahaan.

In [44]:
import numpy as np
import math
import csv

# Cari Perbandingan Berpasangan

In [45]:
def perbandingan(nama, c):
    n_c = len(c)
    perbandingan = np.ones((n_c, n_c))

    print('Tentukan Prioritas dari kriteria-kriteria berikut:')
    for row in range(n_c-1):
        for col in range(row+1, n_c):
            print('Mana yang lebih penting, 1:', c[row],'2:',c[col])

            while True:
                penting = int(input())
                if penting==1 or penting==2:
                    break
                else:
                    print('input salah')

            while True:
                tmp = int(input('Berapa kali lebih penting?(isi dengan bilangan antara 2-9)'))
                if tmp>2 and tmp<9:
                    break
                else:
                    print('input salah')

            if penting==1:
                perbandingan[row,col] = tmp
                perbandingan[col,row] = 1/perbandingan[row,col]

            if penting==2:
                perbandingan[col,row] = tmp
                perbandingan[row,col] = 1/perbandingan[col,row]
    
    path = 'C:/Users/KingMaldio/Desktop/Final Project SPK/' + nama + '.csv'
    with open(path, 'w+') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(perbandingan)
    
    return perbandingan

# AHP

In [23]:
def ahp(ahp):
    norm = normalize_ahp(ahp)
    weight = eigen_vector(norm)
    consistent = consistency_check(norm, weight)
    
    if consistent:
        return weight
    else: print("Matriks AHP tidak konsisten")

In [24]:
def normalize_ahp(ahp):
    norm = []
    
    for row in range(len(ahp)):
        for col in range(ahp[0].shape[0]):
            sums = sum(ahp[:,col])
            norm.append(ahp[row][col]/sums)

    norm = np.asarray(norm)
    norm = norm.reshape(ahp.shape)

    return norm

In [25]:
def eigen_vector(norm):
    weight = []
    
    for row in range(len(norm)):
        sums = sum(norm[row,:])
        weight.append(sums/len(norm))
    
    weight = np.asarray(weight)
    return weight

In [26]:
def consistency_check(norm, weight):
    ri = [0,0,0.58,0.9,1.12,1.24,1.32,1.41,1.45,1.49,1.51,1.48,1.56,1.57,1.59]

    dot = np.dot(norm, weight)
    
    for i in range(len(dot)):
        dot[i] = dot[i]/weight[i]
    
    lamda = sum(dot)/len(dot)
    
    ci = lamda-len(dot)/(len(dot)-1)
    batas = ri[len(dot)]
    cr = ci/batas
    
    if cr>0.1:
        return False
    
    return True

# TOPSIS

In [27]:
def topsis(nilai, weight, flag):
    norm = normalize_topsis(nilai)
    weighted_norm = weighting(norm, weight)
    
    ideal_pos, ideal_neg = ideal(weighted_norm, flag)

    d_pos = dist(weighted_norm, ideal_pos)
    d_neg = dist(weighted_norm, ideal_neg)

    rating = v(d_pos, d_neg)
    
    return rating

In [28]:
def normalize_topsis(nilai):
    tmp = []
    tmp = nilai**2
    norm = np.zeros(nilai.shape)
    
    for col in range(nilai[0].shape[0]):
        sums = sum(tmp[:,col])
        akar = math.sqrt(sums)
        for row in range(len(nilai)):
            norm[row,col] = nilai[row,col]/akar
            
    return norm

In [29]:
def weighting(norm, weight):
    weighted = np.zeros(norm.shape)
    
    for row in range(len(norm)):
        for col in range(norm[0].shape[0]):
            weighted[row,col] = norm[row,col] * weight[col]
    
    return weighted

In [30]:
def ideal(w_norm, flag):
    ideal_pos = np.zeros(flag.shape)
    ideal_neg = np.zeros(flag.shape)
    
    for i in range(len(flag)):
        if flag[i]==1:
            ideal_pos[i] =  max(w_norm[:,i])
            ideal_neg[i] =  min(w_norm[:,i])
        if flag[i]==0:
            ideal_pos[i] =  min(w_norm[:,i])
            ideal_neg[i] =  max(w_norm[:,i])
    return ideal_pos, ideal_neg

In [31]:
def dist(w_norm, ideal):
    diff = np.zeros(w_norm.shape)
    distance = np.zeros(len(w_norm))

    for row in range(len(w_norm)):
        for col in range(w_norm[0].shape[0]):
            tmp = w_norm[row,col] - ideal[col]
            diff[row,col] = tmp**2

        sums = sum(diff[row,:])
        distance[row] = math.sqrt(sums)

    return distance

In [32]:
def v(d_pos, d_neg):
    v = np.zeros(d_pos.shape)
    
    for i in range(len(d_pos)):
        v[i] = d_neg[i]/(d_neg[i]+d_pos[i])
    
    return v

# SAW GDSS

In [33]:
def combine(rating, dm_weight):
    res = (rating[0]*dm_weight[0]+rating[1]*dm_weight[1])*100
    rank = ranking(res)
    
    rank, no_kandidat, hasil = sort(rank, res)
    
    rekomendasi = [*rank, *no_kandidat, *hasil]
    rekomendasi = np.asarray(rekomendasi)
    rekomendasi = rekomendasi.reshape(-1, len(rank)).transpose()
    
    return rekomendasi

In [34]:
def ranking(res):
    rank = np.ones(len(res))
    
    for i in range(len(res)):
        if i==len(res):
            break
            
        for j in range(i+1, len(res)):
            if res[i]<res[j]:
                rank[i]+=1
            else: 
                rank[j]+=1
            
    return rank

In [35]:
def sort(rank, res):
    no_kandidat = np.arange(1, len(rank)+1)
    for i in range(len(rank)):
        if i==len(rank):
            break
            
        for j in range(i+1, len(rank)):
            if rank[i]>rank[j]:
                rank[i], rank[j] = rank[j], rank[i]
                no_kandidat[i], no_kandidat[j] = no_kandidat[j], no_kandidat[i]
                res[i], res[j] = res[j], res[i]
    
    return rank, no_kandidat, res

In [36]:
def penjelasan(rekomendasi):
    print("Berdasarkan perhitungan, Sistem merekomendasikan kandidat dengan urutan sebagai berikut:")
    for row in rekomendasi:
        print(int(row[0]), ':\tkandidat no. ', int(row[1]), 'dengan presentase:', float("{:.2f}".format(row[2])))

# Input User

In [37]:
def import_data(path):
    data = np.genfromtxt(path, delimiter=',')
    data = np.delete(data,(0), axis=0)
    data = np.delete(data,(0), axis=1)
    
    return data

In [38]:
# input nilai wawancara dan bahan AHP
x_nilai = "C:/Users/KingMaldio/Desktop/Final Project SPK/nilai random tes wawancara.csv"
x_flag = "C:/Users/KingMaldio/Desktop/Final Project SPK/flag.csv"
nilai = import_data(x_nilai)
flag = import_data(x_flag)
flag = flag.flatten()

In [41]:
x = input('apakah sudah memiliki data perbandingan kriteria?(y/n)')
while True:
    if x=='y':
        x_ahp_bos = "C:/Users/KingMaldio/Desktop/Final Project SPK/matriks ahp bos.csv"
        x_ahp_hrd = "C:/Users/KingMaldio/Desktop/Final Project SPK/matriks ahp hrd.csv"
        ahp_bos = import_data(x_ahp_bos)
        ahp_hrd = import_data(x_ahp_hrd)
        break
        
    elif x=='n':
        c = ['Cara bicara', 'Etika', 'Semangat', 'Permintaan gaji', 'Wibawa',
             'Pengalaman kerja','Cara berpenampilan', 'Besar motivasi untuk diterima',
             'Perencanaan kehidupan', 'Pengetahuan tentang perusahaan']
        print('Preferensi Direktur:')
        ahp_bos = perbandingan(c)
        ahp_bos = np.asarray(ahp_bos)
        print('Preferensi Manajer HRD:')
        ahp_hrd = perbandingan(c)
        ahp_hrd = np.asarray(ahp_hrd)
        break
    
    else:
        print('input salah')

apakah sudah memiliki data perbandingan kriteria?(y/n)y


# Main Program

In [42]:
# urutan = bos, hrd
dm_weight = [0.6, 0.4]

w = []
rating = []

w.append(ahp(ahp_bos))
w.append(ahp(ahp_hrd))

rating.append(topsis(nilai, w[0], flag))
rating.append(topsis(nilai, w[1], flag))

rekomendasi = combine(rating, dm_weight)

penjelasan(rekomendasi)

Berdasarkan perhitungan, Sistem merekomendasikan kandidat dengan urutan sebagai berikut:
1 :	kandidat no.  4 dengan presentase: 74.31
2 :	kandidat no.  2 dengan presentase: 56.89
3 :	kandidat no.  1 dengan presentase: 38.74
4 :	kandidat no.  3 dengan presentase: 27.9
5 :	kandidat no.  5 dengan presentase: 24.52
